In [1]:
import numpy as np
from pymatgen.core import Structure

def tile_structure(lattice, species, coordinates, scale):
    """
    Tiles a structure by a given scale factor, effectively creating scale^3 more points.

    Parameters:
    - lattice: np.array, the lattice vectors of the structure.
    - species: list, the species (types) of atoms in the structure.
    - coordinates: np.array, the fractional coordinates of the atoms in the structure.
    - scale: int, the scale factor to tile the lattice and coordinates.

    Returns:
    - tiled_lattice: np.array, the scaled lattice vectors.
    - tiled_species: list, the species list repeated scale^3 times.
    - tiled_coordinates: np.array, the tiled coordinates of the atoms.
    """
    # Scale the lattice by the scale factor
    tiled_lattice = lattice * scale * 0.75
    
    # Initialize list to hold the tiled coordinates
    tiled_coordinates_list = []
    
    # Initialize list to hold the tiled species
    tiled_species = []
    
    # Tile the structure by creating additional points in the scaled volume
    for i in range(scale):
        for j in range(scale):
            for k in range(scale):
                # Calculate new coordinates by adding the offset for tiling
                new_coords = (coordinates + np.array([i, j, k])) / scale
                
                # Append the new coordinates to the list
                tiled_coordinates_list.append(new_coords)
                
                # Repeat the species list for each set of new coordinates
                tiled_species.extend(species)
    
    # Concatenate all coordinate arrays into a single NumPy array
    tiled_coordinates = np.concatenate(tiled_coordinates_list, axis=0)
    
    return tiled_lattice, tiled_species, tiled_coordinates

In [2]:
from diffusion.inference.process_generated_crystals import (
    get_one_crystal,
    load_sample_results_from_hdf5,
)
from diffusion.inference.visualize_crystal import plot_crystal
import crystal_toolkit

OUT_DIR = "out"
crystal_file = f"{OUT_DIR}/crystals.h5"
#crystal_file = f"{OUT_DIR}/relax/relaxed.h5"
sample_result = load_sample_results_from_hdf5(crystal_file)

def show_crystal(i, tile_amount=1):
    lattice, frac_x, atomic_numbers = get_one_crystal(sample_result, i)
    #print(lattice)
    frac_x -= np.min(frac_x, axis=0)
    tiled_lattice, tiled_species, tiled_coordinates = tile_structure(lattice, atomic_numbers, frac_x, tile_amount)
    display(Structure(tiled_lattice, tiled_species, tiled_coordinates))

#fig = plot_crystal(atomic_numbers, lattice, frac_x, show_bonds=False)
#display(fig)

/Users/curtischong/Documents/dev/arreau/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
show_crystal(0)

In [4]:
show_crystal(1)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [5]:
show_crystal(2)

In [27]:
show_crystal(3)

In [28]:
show_crystal(4)

In [29]:
show_crystal(5)

In [30]:
show_crystal(6)

In [31]:
show_crystal(7)

In [32]:
show_crystal(8)

In [33]:
show_crystal(9)

In [34]:
show_crystal(10)

In [35]:
show_crystal(11)

In [44]:
show_crystal(12, 3)

In [37]:
show_crystal(13)

In [38]:
show_crystal(14)

In [39]:
show_crystal(15)

In [40]:
show_crystal(16)

In [41]:
show_crystal(17)

In [42]:
show_crystal(18)

In [43]:
show_crystal(19)